# 20th Century Fox 2000 - 2020 All Movies Data

#### Scraping the main page to get the links for all individual movies

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

main_site = requests.get("https://en.wikipedia.org/wiki/List_of_20th_Century_Fox_films_(2000%E2%80%932020)")
souped_movie_main = bs(main_site.content,"html.parser")
main_table = souped_movie_main.find_all("table",class_="wikitable sortable")
link_list = []
for val in range(0,len(main_table)):
    main_table_content = main_table[val].tbody.find_all("tr")
    for val1 in range(1,len(main_table_content)):
        link_list.append(main_table_content[val1].a.get("href"))

#### Function to return the text values as list if many are present or as a string if idividual

In [ ]:
def get_content(content):
    if len(content.find_all("li")) > 0:
        # If there are multiple values corresponding to one key
        lis = [li.text.replace("\xa0"," ").replace("[1]","") for li in content.find_all("li")]
        return lis
    else:
        # If there is only one Value corresponding to a key
        lis = (content.td.text).replace("\xa0"," ").replace("[1]","")
        return lis

#### From the main page, each url is again passed and the infobox data is scraped and stored in a dictionary where keys are Movie Title, Director, Budget etc. and values are the values corresponding to the keys in infobox 

In [ ]:
full_movie_list = []
for url in link_list:
    try:
        site = requests.get("https://en.wikipedia.org/"+url)
        souped_movie = bs(site.content,"html.parser")

        table = souped_movie.find("table",class_="infobox vevent")
        table_content = table.find_all("tr")
        movie_dict = {}
        movie_dict['Title'] = table_content[0].text
        for x in range(2,len(table_content)):
            movie_dict[table_content[x].th.text] = get_content(table_content[x])
        # A list of dictionaries is created this way
        full_movie_list.append(movie_dict)
    except:
        print("An Exception Occured at url " + url)
        
# dictionary is directly converted to DataFrame where the keys form the columns
df = pd.DataFrame(full_movie_list)